## Voy a probar usando superficie cubierta y superficie total

In [ ]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn.externals import joblib
import time

import warnings
warnings.simplefilter("ignore", category=DeprecationWarning)

In [ ]:
propiedades = pd.read_csv('../../???.csv')

sup_min = 10
sup_max = 500  #VER#################################
lat_min = -35
lat_max = -34
lon_min = -59
lon_max = -58
precio_min = 30000
precio_max = 3000000

propiedades = propiedades.loc[(propiedades.price_aprox_usd.notnull()) \
                              & (propiedades.lat.notnull()) & (propiedades.lon.notnull()) & (propiedades.Year >=2017)\
                        & (propiedades.price_aprox_usd <= precio_max) & (propiedades.price_aprox_usd >= precio_min) &\
                         (propiedades.lat <= lat_max) & (propiedades.lat >= lat_min) &\
                       (propiedades.lon <= lon_max) & (propiedades.lon >= lon_min),:]

In [ ]:
set_pruebas = pd.read_csv('../properati_dataset_modificado2.csv')
set_pruebas.loc[:,'price_usd'] = 0

def normalizar_sup(s):
    if not s:
        return None
    if s < sup_min:
        return sup_min
    if s > sup_max:
        return sup_max
    return s

set_pruebas.loc[:,'surface_covered_in_m2'] = set_pruebas.loc[:,'surface_covered_in_m2'].apply(lambda x: normalizar_sup(x))
set_pruebas.loc[:,'surface_total_in_m2'] = set_pruebas.loc[:,'surface_total_in_m2'].apply(lambda x: normalizar_sup(x))

# Calculo para aquellos que tienen tanto superficie total como cubierta

In [ ]:
condiciones = (set_pruebas.surface_covered_in_m2.notnull()) & (set_pruebas.surface_total_in_m2.notnull())
cond_prop = (propiedades.surface_covered_in_m2.notnull()) & (propiedades.surface_total_in_m2.notnull()) &\
                    (propiedades.surface_covered_in_m2 <= sup_max) & (propiedades.surface_covered_in_m2 >= sup_min) &\
            (propiedades.surface_total_in_m2 <= sup_max) & (propiedades.surface_total_in_m2 >= sup_min) 


In [ ]:
cant = 15
col = ['surface_covered_in_m2','surface_total_in_m2','lat', 'lon','property_type_encoded']
comodidades = ['seguridad','gimnasio','aire','pileta','cochera']

for i in range(cant):
    com = list(np.random.choice(comodidades,4,replace = False))
    columnas = col + com
    set_entrenamiento_datos = propiedades.loc[cond_prop,columnas]
    set_entrenamiento_resultado = propiedades.loc[cond_prop,'price_aprox_usd']
    
    tree = DecisionTreeRegressor(criterion = 'mse', max_depth = 100, max_features = 0.8)
    
    tree.fit(set_entrenamiento_datos,set_entrenamiento_resultado)
    set_pruebas.loc[condiciones,'price_usd'] = set_pruebas.loc[condiciones,'price_usd'] + \
            set_pruebas.loc[condiciones,columnas].apply(lambda x: tree.predict([x])[0],axis = 1)
    print(i,end = '-')

set_pruebas.loc[condiciones,'price_usd'] = set_pruebas.loc[condiciones,'price_usd'] / cant

# Calculo para aquellos que solo tienen superficie cubierta

In [ ]:
condiciones = (set_pruebas.surface_covered_in_m2.notnull()) & (set_pruebas.price_usd == 0)
cond_prop = (propiedades.surface_covered_in_m2.notnull()) &\
        (propiedades.surface_covered_in_m2 <= sup_max) & (propiedades.surface_covered_in_m2 >= sup_min)

In [ ]:
cant = 15
col = ['surface_covered_in_m2','lat', 'lon','property_type_encoded']
comodidades = ['seguridad','gimnasio','aire','pileta','cochera']

for i in range(cant):
    com = list(np.random.choice(comodidades,4,replace = False))
    columnas = col + com
    set_entrenamiento_datos = propiedades.loc[cond_prop,columnas]
    set_entrenamiento_resultado = propiedades.loc[cond_prop,'price_aprox_usd']
    
    tree = DecisionTreeRegressor(criterion = 'mse', max_depth = 100, max_features = 0.8)
    
    tree.fit(set_entrenamiento_datos,set_entrenamiento_resultado)
    set_pruebas.loc[condiciones,'price_usd'] = set_pruebas.loc[condiciones,'price_usd'] + \
            set_pruebas.loc[condiciones,columnas].apply(lambda x: tree.predict([x])[0],axis = 1)
    print(i,end = '-')

set_pruebas.loc[condiciones,'price_usd'] = set_pruebas.loc[condiciones,'price_usd'] / cant

# Calculo para aquellos que solo tienen superficie total

In [ ]:
condiciones = (set_pruebas.surface_total_in_m2.notnull()) & (set_pruebas.price_usd == 0)
cond_prop = (propiedades.surface_total_in_m2.notnull()) &\
            (propiedades.surface_total_in_m2 <= sup_max) & (propiedades.surface_total_in_m2 >= sup_min)

In [ ]:
cant = 15
col = ['surface_total_in_m2','lat', 'lon','property_type_encoded']
comodidades = ['seguridad','gimnasio','aire','pileta','cochera']

for i in range(cant):
    com = list(np.random.choice(comodidades,4,replace = False))
    columnas = col + com
    set_entrenamiento_datos = propiedades.loc[cond_prop,columnas]
    set_entrenamiento_resultado = propiedades.loc[cond_prop,'price_aprox_usd']
    
    tree = DecisionTreeRegressor(criterion = 'mse', max_depth = 100, max_features = 0.8)
    
    tree.fit(set_entrenamiento_datos,set_entrenamiento_resultado)
    set_pruebas.loc[condiciones,'price_usd'] = set_pruebas.loc[condiciones,'price_usd'] + \
            set_pruebas.loc[condiciones,columnas].apply(lambda x: tree.predict([x])[0],axis = 1)
    print(i,end = '-')

set_pruebas.loc[condiciones,'price_usd'] = set_pruebas.loc[condiciones,'price_usd'] / cant

# Calculo para aquellos que no tienen superficie

In [ ]:
condiciones =  (set_pruebas.price_usd == 0)

In [ ]:
cant = 15
col = ['lat', 'lon','property_type_encoded']
comodidades = ['seguridad','gimnasio','aire','pileta','cochera']

for i in range(cant):
    com = list(np.random.choice(comodidades,4,replace = False))
    columnas = col + com
    set_entrenamiento_datos = propiedades.loc[:,columnas]
    set_entrenamiento_resultado = propiedades.loc[:,'price_aprox_usd']
    
    tree = DecisionTreeRegressor(criterion = 'mse', max_depth = 100, max_features = 0.8)
    
    tree.fit(set_entrenamiento_datos,set_entrenamiento_resultado)
    set_pruebas.loc[condiciones,'price_usd'] = set_pruebas.loc[condiciones,'price_usd'] + \
            set_pruebas.loc[condiciones,columnas].apply(lambda x: tree.predict([x])[0],axis = 1)
    print(i,end = '-')

set_pruebas.loc[condiciones,'price_usd'] = set_pruebas.loc[condiciones,'price_usd'] / cant

### Resultado final

In [ ]:
resultado = set_pruebas.loc[:,['id','price_usd']]

resultado.to_csv('resultados/Decision_tree_superficies.csv', index = False)
resultado.describe()

# ....................................................
# ....................................................

In [ ]:
a = pd.read_csv("../properati_dataset_modificado.csv")
set_pruebas = pd.read_csv("../properati_dataset_modificado2.csv")
def normalizar_sup(s):
    if s < sup_min:
        return sup_min
    if s > sup_max:
        return sup_max
    return s
sup_min = 10
sup_max = 500
lat_min = -35
lat_max = -34
lon_min = -59
lon_max = -58
precio_min = 30000
precio_max = 3000000

set_pruebas.loc[:,'superficie'] = a.loc[:,'superficie'].apply(lambda x: normalizar_sup(x))
set_pruebas.loc[:,'superficie'].describe()

In [ ]:
propiedades = pd.read_csv('../../datos_con_fecha.csv')

sup_min = 10
sup_max = 500
lat_min = -35
lat_max = -34
lon_min = -59
lon_max = -58
precio_min = 30000
precio_max = 3000000

propiedades = propiedades.loc[(propiedades.price_aprox_usd.notnull()) & (propiedades.superficie.notnull())\
                              & (propiedades.lat.notnull()) & (propiedades.lon.notnull()) & (propiedades.Year >=2017)\
                        & (propiedades.price_aprox_usd <= precio_max) & (propiedades.price_aprox_usd >= precio_min) &\
                         (propiedades.superficie <= sup_max) & (propiedades.superficie >= sup_min) &\
                      (propiedades.lat <= lat_max) & (propiedades.lat >= lat_min) &\
                       (propiedades.lon <= lon_max) & (propiedades.lon >= lon_min),:]

set_pruebas.loc[:,'price_usd'] = 0.0
cant = 10
col = ['superficie','lat', 'lon','property_type_encoded']
comodidades = ['seguridad','gimnasio','aire','pileta','cochera']

for i in range(cant):
    com = list(np.random.choice(comodidades,4,replace = False))
    columnas = col + com
    set_entrenamiento_datos = propiedades.loc[:,columnas]
    set_entrenamiento_resultado = propiedades.loc[:,'price_aprox_usd']
    
    tree = DecisionTreeRegressor(criterion = 'mse', max_depth = 100, max_features = 0.8)
    
    tree.fit(set_entrenamiento_datos,set_entrenamiento_resultado)
    set_pruebas.loc[:,'res'] = set_pruebas.loc[:,columnas].apply(lambda x: tree.predict([x])[0],axis = 1)
    set_pruebas.loc[:,'price_usd'] = set_pruebas.loc[:,'price_usd'] + set_pruebas.loc[:,'res']
    print(i,end = '-')
set_pruebas.loc[:,'price_usd'] = set_pruebas.loc[:,'price_usd'] / cant

resultado = set_pruebas.loc[:,['id','price_usd']]

resultado.to_csv('resultados/Decision_tree_menosColumnas_sup500.csv', index = False)